# Importing Dependencies

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic #model
mp_drawing = mp.solutions.drawing_utils #drawing stuff

In [4]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert Colour
    image.flags.writeable = False # Image not writeable
    results = model.process(image) # Make Prediction
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image,results

# We do the conversion as cv2 needs to be in an RGB colour space to work by default and we save a little memory

In [5]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [6]:
def draw_styled_landmarks(image,results):
    # Draw connections
    mp_drawing.draw_landmarks(image,results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                              mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1))
    
    mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2))
    
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(121,22,76),thickness=1,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250),thickness=1,circle_radius=2))
    
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(245,117,66),thickness=1,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=1,circle_radius=2))

In [12]:
cap = cv2.VideoCapture(0)

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame = cap.read() #Reads the feed
    
        image, results = mediapipe_detection(frame,holistic) #detection stuff

        draw_styled_landmarks(image,results) # Draw landmarks in realtime
        
        cv2.imshow('OpenCV Feed',image) 
    
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
      
    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1718514340.890082  775684 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1718514340.968742  779157 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718514340.974878  779157 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718514340.975531  779157 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718514340.975534  779162 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718514340.975760  779156 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp

# Extracting Keypoint Values

## Extracting landmarks and putting them in a numpy array
If they are empty, replace with numpy zeros

In [40]:
# lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

In [41]:
# rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)

In [42]:
# face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(1404)

In [43]:
# pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(132)

In [53]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [54]:
extract_keypoints(results).shape

(1662,)

# Setup Folders for Collection

In [57]:
#Path for exported data and numpy arrays
DATA_PATH = os.path.join('MP_DATA')

#Actions to detect
actions = np.array(['hello', 'thanks', 'ily'])

no_sequences = 30 #vids
sequence_length = 30 #frames with 1662 keypoints

# 3 actions, 30 frames per action, 30 videos, 1662 keypoints

In [58]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

# Collect the Data